In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [2]:
# Calculate the total number of schools
total_schools = len(school_data_complete["School ID"].unique())
total_schools

15

In [3]:
# Calculate the total number of students
total_students = len(school_data_complete["Student ID"].unique())
total_students

39170

In [4]:
# Calculate the total budget
total_budget = school_data["budget"].sum()
total_budget

24649428

In [5]:
# Calculate the average math score
avg_math_score = school_data_complete["math_score"].mean()
avg_math_score

78.98537145774827

In [6]:
# Calculate the average reading score
avg_reading_score = school_data_complete["reading_score"].mean()
avg_reading_score

81.87784018381414

In [7]:
# Create bins to place score values
bins = [0, 69, 100]

# Create labels for these bins
score_labels = ["Fail", "Pass"]

In [8]:
# Place the sliced bin data into new columns inside of the data, for math and reading
school_data_complete["Math Grade Group"] = pd.cut(school_data_complete["math_score"], bins, labels=score_labels)
school_data_complete["Reading Grade Group"] = pd.cut(school_data_complete["reading_score"], bins, labels=score_labels)
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,Math Grade Group,Reading Grade Group
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,Pass,Fail
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,Fail,Pass
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,Fail,Pass
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,Fail,Fail
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,Pass,Pass


In [9]:
# Isolate passing scores for math
pass_math = school_data_complete.loc[(
    school_data_complete["Math Grade Group"] == "Pass")]

# Calculate the percentage of students with a passing math score (70 or greater)
perc_pass_math = (pass_math["Student ID"].count() / total_students) * 100
perc_pass_math

74.9808526933878

In [10]:
# Isolate passing scores for reading
pass_reading = school_data_complete.loc[(
    school_data_complete["Reading Grade Group"] == "Pass")]

# Calculate the percentage of students with a passing reading score (70 or greater)
perc_pass_reading = (pass_reading["Student ID"].count() / total_students) * 100
perc_pass_reading

85.80546336482001

In [11]:
# Isolate overall passing scores, reading and math
pass_overall = school_data_complete.loc[(
    school_data_complete["Math Grade Group"] == "Pass") & 
    (school_data_complete["Reading Grade Group"] == "Pass")]

# Calculate the percentage of students who passed math and reading (% Overall Passing)
perc_pass_overall = (pass_overall["Student ID"].count() / total_students) * 100
perc_pass_overall

65.17232575950983

In [12]:
# Create a dataframe to hold the above results
district_summary_df = pd.DataFrame({
    "Total Schools": [total_schools], "Total Students": [total_students],
    "Total Budget": [total_budget], "Average Math Score": [avg_math_score],
    "Average Reading Score": [avg_reading_score], "% Passing Math": [perc_pass_math],
    "% Passing Reading": [perc_pass_reading], "% Overall Passing": [perc_pass_overall]
})
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [13]:
# Optional: give the displayed data cleaner formatting
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${0:,.2f}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.2f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.2f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.2f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.2f}".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.2f}".format)
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


In [14]:
# Using GroupBy in order to separate the data into fields according to different schools
grouped_school_df = school_data_complete.groupby(["school_name"])
grouped_school_df.count()

,Student ID,student_name,gender,grade,reading_score,math_score,School ID,type,size,budget,Math Grade Group,Reading Grade Group
school_name,,,,,,,,,,,,
Bailey High School,4976,4976,4976,4976,4976,4976,4976,4976,4976,4976,4976,4976
Cabrera High School,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858
Figueroa High School,2949,2949,2949,2949,2949,2949,2949,2949,2949,2949,2949,2949
Ford High School,2739,2739,2739,2739,2739,2739,2739,2739,2739,2739,2739,2739
Griffin High School,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468
Hernandez High School,4635,4635,4635,4635,4635,4635,4635,4635,4635,4635,4635,4635
Holden High School,427,427,427,427,427,427,427,427,427,427,427,427
Huang High School,2917,2917,2917,2917,2917,2917,2917,2917,2917,2917,2917,2917
Johnson High School,4761,4761,4761,4761,4761,4761,4761,4761,4761,4761,4761,4761


In [15]:
# Find School Type
school_type = grouped_school_df["type"].unique()
school_type

school_name
Bailey High School       [District]
Cabrera High School       [Charter]
Figueroa High School     [District]
Ford High School         [District]
Griffin High School       [Charter]
Hernandez High School    [District]
Holden High School        [Charter]
Huang High School        [District]
Johnson High School      [District]
Pena High School          [Charter]
Rodriguez High School    [District]
Shelton High School       [Charter]
Thomas High School        [Charter]
Wilson High School        [Charter]
Wright High School        [Charter]
Name: type, dtype: object

In [16]:
# Find total students per school
school_student_count = grouped_school_df["Student ID"].count()
school_student_count.head()

school_name
Bailey High School      4976
Cabrera High School     1858
Figueroa High School    2949
Ford High School        2739
Griffin High School     1468
Name: Student ID, dtype: int64

In [17]:
# Find Total School Budget
school_budget = grouped_school_df["budget"].mean()
school_budget

school_name
Bailey High School       3124928
Cabrera High School      1081356
Figueroa High School     1884411
Ford High School         1763916
Griffin High School       917500
Hernandez High School    3022020
Holden High School        248087
Huang High School        1910635
Johnson High School      3094650
Pena High School          585858
Rodriguez High School    2547363
Shelton High School      1056600
Thomas High School       1043130
Wilson High School       1319574
Wright High School       1049400
Name: budget, dtype: int64

In [18]:
# Find Per Student Budget
per_student_budget = school_budget / school_student_count
per_student_budget

school_name
Bailey High School       628.0
Cabrera High School      582.0
Figueroa High School     639.0
Ford High School         644.0
Griffin High School      625.0
Hernandez High School    652.0
Holden High School       581.0
Huang High School        655.0
Johnson High School      650.0
Pena High School         609.0
Rodriguez High School    637.0
Shelton High School      600.0
Thomas High School       638.0
Wilson High School       578.0
Wright High School       583.0
dtype: float64

In [19]:
# Find Average Math Score
school_avg_math_score = grouped_school_df["math_score"].mean()
school_avg_math_score

school_name
Bailey High School       77.048432
Cabrera High School      83.061895
Figueroa High School     76.711767
Ford High School         77.102592
Griffin High School      83.351499
Hernandez High School    77.289752
Holden High School       83.803279
Huang High School        76.629414
Johnson High School      77.072464
Pena High School         83.839917
Rodriguez High School    76.842711
Shelton High School      83.359455
Thomas High School       83.418349
Wilson High School       83.274201
Wright High School       83.682222
Name: math_score, dtype: float64

In [20]:
# Find Average Reading Score
school_avg_reading_score = grouped_school_df["reading_score"].mean()
school_avg_reading_score

school_name
Bailey High School       81.033963
Cabrera High School      83.975780
Figueroa High School     81.158020
Ford High School         80.746258
Griffin High School      83.816757
Hernandez High School    80.934412
Holden High School       83.814988
Huang High School        81.182722
Johnson High School      80.966394
Pena High School         84.044699
Rodriguez High School    80.744686
Shelton High School      83.725724
Thomas High School       83.848930
Wilson High School       83.989488
Wright High School       83.955000
Name: reading_score, dtype: float64

In [47]:
# Count how many of pass values for each school and create DataFrame
pass_math_df = pd.DataFrame(grouped_school_df["Math Grade Group"].value_counts())

# Rename the "Math Grade Group" column to "Math Counts"
pass_math_df = pass_math_df.rename(
    columns={"Math Grade Group": "Math Counts"})

# Reset the index of "pass_math_df" so that includes all columns
pass_math_df = pass_math_df.reset_index()

# Isolate passing scores for math
sch_pass_math = pass_math_df.loc[(
    pass_math_df["Math Grade Group"] == "Pass")]

# Merge the "school_student_count" into the DataFrame
sch_pass_math = sch_pass_math.merge(school_student_count, on="school_name")

# Add column for % passed
sch_pass_math["% Passed Math"] = (sch_pass_math["Math Counts"] / sch_pass_math["Student ID"]) * 100
sch_pass_math

,school_name,Math Grade Group,Math Counts,Student ID,% Passed Math
0,Bailey High School,Pass,3318,4976,66.680064
1,Cabrera High School,Pass,1749,1858,94.133477
2,Figueroa High School,Pass,1946,2949,65.988471
3,Ford High School,Pass,1871,2739,68.309602
4,Griffin High School,Pass,1371,1468,93.392371
5,Hernandez High School,Pass,3094,4635,66.752967
6,Holden High School,Pass,395,427,92.505855
7,Huang High School,Pass,1916,2917,65.683922
8,Johnson High School,Pass,3145,4761,66.057551
9,Pena High School,Pass,910,962,94.594595


In [48]:
# Count how many of pass values for each school and create DataFrame, for reading
pass_rdg_df = pd.DataFrame(grouped_school_df["Reading Grade Group"].value_counts())

# Rename the "Reading Grade Group" column to "Reading Counts"
pass_rdg_df = pass_rdg_df.rename(
    columns={"Reading Grade Group": "Reading Counts"})

# Reset the index of "pass_rdg_df" so that includes all columns
pass_rdg_df = pass_rdg_df.reset_index()

# Isolate passing scores for reading
sch_pass_rdg = pass_rdg_df.loc[(
    pass_rdg_df["Reading Grade Group"] == "Pass")]

# Merge the "school_student_count" into the DataFrame
sch_pass_rdg = sch_pass_rdg.merge(school_student_count, on="school_name")

# Add column for % passed
sch_pass_rdg["% Passed Rdg"] = (sch_pass_rdg["Reading Counts"] / sch_pass_rdg["Student ID"]) * 100
sch_pass_rdg

,school_name,Reading Grade Group,Reading Counts,Student ID,% Passed Rdg
0,Bailey High School,Pass,4077,4976,81.933280
1,Cabrera High School,Pass,1803,1858,97.039828
2,Figueroa High School,Pass,2381,2949,80.739234
3,Ford High School,Pass,2172,2739,79.299014
4,Griffin High School,Pass,1426,1468,97.138965
5,Hernandez High School,Pass,3748,4635,80.862999
6,Holden High School,Pass,411,427,96.252927
7,Huang High School,Pass,2372,2917,81.316421
8,Johnson High School,Pass,3867,4761,81.222432
9,Pena High School,Pass,923,962,95.945946
